<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/05_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
#| default_exp export

# Exporting Notebooks to MD
> Exporting notebooks to markdown for use in blog posts and W&B documentation

In [ ]:
#| export
import os
from pathlib import Path
from rich.markdown import Markdown
from fastcore.script import call_parse, Param, store_true

from nb_helpers.colab import has_colab_badge, get_colab_url
from nb_helpers.utils import find_nbs, read_nb, is_nb

In [ ]:
test_nb = Path("test_data/test_nb.ipynb")
notebook = read_nb(test_nb)

In [ ]:
has_colab_badge(notebook)

0

In [ ]:
cells = notebook["cells"]

In [ ]:
cells

[{'cell_type': 'markdown',
  'metadata': {},
  'source': '<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/test_data/test_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>\n<!--- @wandbcode{tracker} -->',
  'idx_': 0},
 {'cell_type': 'markdown',
  'metadata': {},
  'source': '# This is a Test NB \n<!--- @wandbcode{tracker} -->',
  'idx_': 1},
 {'cell_type': 'markdown',
  'metadata': {},
  'source': 'this notebooks has code and text and this fancy walala',
  'idx_': 2},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': 'import os, sys\nimport logging as log\nfrom pathlib import Path\nfrom fastcore.basics import ifnone\n\ndef f(x): \n    return x',
  'idx_': 3},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [],
  'source': 'f(2)',
  'idx_': 4},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata'

In [ ]:
#| export
def _export_code_cell(source):
    "Wrap code around python"
    return f"\n```python\n{source}\n```\n\n"

def default_line_filter(line):
    if "colab.research" in line:
        return False
    elif "@wandbcode" in line:
        return False
    else: 
        return True

def filter_out_lines(source, f=default_line_filter):
    "Filter lines from source with filter f"
    lines = source.split("\n")
    return "\n".join(filter(f, lines))

def _export_md_cell(source):
    source = filter_out_lines(source)
    return f"{source} \n"

In [ ]:
#| export
def export_cell(cell, debug=False):
    "Export cell source to string"
    source = cell["source"]
    if debug:
        print(source)
    if cell["cell_type"] == "markdown":
        return _export_md_cell(source)
        
    elif cell["cell_type"] == "code":
        return  _export_code_cell(source)
    else:
        return ""

In [ ]:
exported_file = "test_nb.md"

In [ ]:
export_cells = [export_cell(c) for c in cells]

In [ ]:
#| export
def colab_url(file):
    "Create a fresh colab URL from file, must be on github repo"
    url = get_colab_url(file, branch="master")
    return f'\n[![](https://colab.research.google.com/assets/colab-badge.svg)]({url})\n\n'

In [ ]:
colab_link_md = colab_url(test_nb)

In [ ]:
export_cells

[' \n',
 '# This is a Test NB  \n',
 'this notebooks has code and text and this fancy walala \n',
 '\n```python\nimport os, sys\nimport logging as log\nfrom pathlib import Path\nfrom fastcore.basics import ifnone\n\ndef f(x): \n    return x\n```\n\n',
 '\n```python\nf(2)\n```\n\n',
 '\n```python\nwith open("file.txt", "w") as writer:\n    writer.write(\'one line\')\n```\n\n',
 '\n```python\nfrom itertools import chain\n```\n\n']

In [ ]:
export_cells.insert(2, colab_link_md)

In [ ]:
Markdown("".join(export_cells))

╔═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                This is a Test NB                                                ║
╚═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝

🌆 ]8;id=890167;https://colab.research.google.com/github/wandb/nb_helpers/blob/master/nbs/test_data/test_nb.ipynb\colab-badge.svg]8;;\                                                                                                                    

this notebooks has code and text and this fancy walala                                                             

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ import os, sys                                                                                                  │
│ import logging as log                                                                                           │
│ from pathlib import Path                                                                                        │
│ from fastcore.basics import ifnone                                                                              │
│                                                                                                                 │
│ def f(x):                                                                                                       │
│     return x                                                                                                    │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ f(2)                                                                                                            │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ with open("file.txt", "w") as writer:                                                                           │
│     writer.write('one line')                                                                                    │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ from itertools import chain                                                                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
#| export
@call_parse
def export(
    path: Param("A path to nb files", Path, nargs="?", opt=False) = os.getcwd(),
    verbose: Param("Print errors along the way", store_true) = False,
    outfile: Param("An output file to save") = None,
):
    if is_nb(path):
        nb = read_nb(path)
    else:
        raise Error(f"This {path} is not a notebook!")
    
    if outfile is None:
        outfile = path.with_suffix(".md")
        print(f"Exporting notebook {path} -> {outfile}")
    
    cells = notebook["cells"]
    export_cells = [export_cell(c) for c in cells]
    
    with open(outfile, "w") as f:
        f.writelines(export_cells)

In [ ]:
export(test_nb)

Exporting notebook test_data/test_nb.ipynb -> test_data/test_nb.md
